In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.metrics import confusion_matrix,f1_score

In [28]:
##训练集
data=pd.read_csv("./train_lgb_1.csv")
data=data.values
data_1=pd.read_csv("./train_lgb_3.csv")
data_1=data_1.values
data_2=pd.read_csv("./train_lgb_local.csv")
data_2=data_2.values
data_3=pd.read_csv("./prob_catboost_train.csv")
data_3=data_3.values
data_4=pd.read_csv("./cnn_train_label.csv",header=None)
data_4=data_4.values
train_label=pd.read_csv("./y_train.csv")
train_label=train_label.loc[:,'flag'].values

In [24]:
#阈值选择
#train:0.36
#train_1:0.39
#train_2:0.41
#train_3:0.38
yu_num=np.arange(0.3,0.5,0.01)
for yu in yu_num:
    print("------{}------".format(yu))
    label=[]
    for i in range(data_3.shape[0]):
        if (data_3[i]>yu):
            label.append(1)
        else:
            label.append(0)
    print(confusion_matrix(train_label,label))
    print(f1_score(train_label,label))
    print(accuracy_score(train_label,label))

------0.3------
[[693253   6747]
 [   823  83177]]
0.9564752420597502
0.990344387755102
------0.31------
[[693715   6285]
 [   856  83144]]
0.958824648703504
0.990891581632653
------0.32------
[[694112   5888]
 [   894  83106]]
0.9607963281963536
0.9913494897959184
------0.33------
[[694522   5478]
 [   927  83073]]
0.9628805396665334
0.9918303571428572
------0.34------
[[694917   5083]
 [   981  83019]]
0.9647650811727928
0.992265306122449
------0.35000000000000003------
[[695291   4709]
 [  1017  82983]]
0.9666495818092864
0.9926964285714286
------0.36000000000000004------
[[695602   4398]
 [  1075  82925]]
0.9680544935589501
0.9930191326530612
------0.37000000000000005------
[[695916   4084]
 [  1121  82879]]
0.9695548159543292
0.9933609693877551
------0.38000000000000006------
[[696210   3790]
 [  1165  82835]]
0.9709597069597069
0.9936798469387755
------0.39000000000000007------
[[696487   3513]
 [  1219  82781]]
0.9722127614595933
0.9939642857142857
------0.4000000000000001------

In [30]:
#投票法
#yu=2
yu_num=[1,2,3,4]
for yu in yu_num:
    print("-----{}----".format(yu))
    label=[]
    for i in range(data.shape[0]):
        loc=0
        if (data[i]>0.36):
            loc+=1
        if (data_1[i]>0.39):
            loc+=1
        if (data_2[i]>0.41):
            loc+=1
        if (data_3[i]>0.38):
            loc+=1
        if (data_4[i,0]<0.5):
            loc+=1
        if (loc>yu):
            label.append(1)
        else:
            label.append(0)
    print(confusion_matrix(train_label,label))
    print(f1_score(train_label,label))
    print(accuracy_score(train_label,label))

-----1----
[[699807    193]
 [    12  83988]]
0.9987810751511764
0.9997385204081632
-----2----
[[699965     35]
 [    19  83981]]
0.9996786020379012
0.9999311224489796
-----3----
[[699980     20]
 [   197  83803]]
0.9987069710349594
0.9997232142857143
-----4----
[[700000      0]
 [ 78594   5406]]
0.12093148110865044
0.8997525510204082


In [31]:
#测试集
data=pd.read_csv("./prob_lgb_1.csv")
data_1=pd.read_csv("./prob_lgb_2.csv")
data_2=pd.read_csv("./prob_catboost.csv")
data_3=pd.read_csv("./prob_lgb+focal_loss.csv")
test_label=pd.read_csv("./test_label.csv")
test_label=test_label.values
print(test_label.shape)

(340043, 1)


In [5]:
label=[]
for i in range(data.shape[0]):
    if (data.iloc[i,0]>0.5):
        label.append(1)
    else:
        label.append(0)

In [36]:
###1
label=[]
for i in range(data.shape[0]):
    if (data.iloc[i,0]>0.36):
        label.append(1)
    else:
        label.append(0)

In [37]:
###2
label_1=[]
for i in range(data_1.shape[0]):
    if (data_1.iloc[i,0]>0.39):
        label_1.append(1)
    else:
        label_1.append(0)

In [39]:
###3
label_2=[]
for i in range(data_2.shape[0]):
    if (data_2.iloc[i,0]>0.41):
        label_2.append(1)
    else:
        label_2.append(0)

In [40]:
###4
label_3=[]
for i in range(data_3.shape[0]):
    if (data_3.iloc[i,0]>0.38):
        label_3.append(1)
    else:
        label_3.append(0)

In [8]:
###5
label_cnn=pd.read_csv("./cnn_test_label.csv",header=None)
label_cnn=np.array(label_cnn)

In [72]:
vote_label=[]
for i in range(label_cnn.shape[0]):
    loc=0
    if (label[i]==1):
        loc+=1
    if (label_1[i]==1):
        loc+=1
    if (label_2[i]==1):
        loc+=1
    if (label_3[i]==1):
        loc+=1
    if (label_cnn[i]==1):
        loc+=1
    if (loc>3):
        vote_label.append(1)
    else:
        vote_label.append(0)

In [73]:
confusion_matrix(test_label,vote_label)

array([[294147,   5853],
       [  4456,  35587]], dtype=int64)

In [74]:
print(f1_score(test_label,vote_label))

0.8734828123657695


In [33]:
label_s=[]
for i in range(data.shape[0]):
    loc=0
    loc+=data.iloc[i,0]
    loc+=(data_1.iloc[i,0]+data_2.iloc[i,0]+data_3.iloc[i,0])
    if (loc>1.59):
        label_s.append(1)
    else:
        label_s.append(0)
print(f1_score(test_label,label_s))
print(confusion_matrix(test_label,label_s))

0.8738448104397604
[[293615   6385]
 [  4017  36026]]


In [34]:
label_s = pd.DataFrame(label_s,columns=['flag'])
label_s.to_csv('./final_test_label.csv',index = False)